In [2]:
import pandas as pd
import numpy as np
import json
from pathlib import Path

# --- Импорты для Моделей ---
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# --- Импорты для Экспорта ---
try:
    import skl2onnx
    from skl2onnx import convert_sklearn
    from skl2onnx.common.data_types import FloatTensorType
except ImportError:
    print("Внимание: 'skl2onnx' не установлен. Экспорт в ONNX не сработает.")
    print("Установите: pip install skl2onnx")


# ===================================================================
# --- Шаг 1: Функции подготовки данных ---
# ===================================================================

def _zscore(s: pd.Series, window=7, min_periods=3) -> pd.Series:
    """Расчет Z-оценки для временного ряда, игнорируя std=0."""
    m = s.rolling(window, min_periods=min_periods).mean()
    sd = s.rolling(window, min_periods=min_periods).std()
    sd = sd.replace(0, np.nan) 
    return (s - m) / sd

def _delta_from_mean(s: pd.Series, window=7, min_periods=3) -> pd.Series:
    """Расчет дельты от скользящего среднего."""
    m = s.rolling(window, min_periods=min_periods).mean()
    return s - m

def create_fatigue_model_dataset(health_fitness_dataset: str) -> pd.DataFrame:
    """
    Загружает сырой датасет, очищает его, ИГНОРИРУЯ ПУЛЬС,
    и создает новую, инженерную цель 'y_target_fatigue'.
    """
    
    print(f"--- Загрузка сырых данных из {health_fitness_dataset} ---")
    try:
        # --- ИСПРАВЛЕНО: Ошибка 1 (убрано .csv) ---
        df = pd.read_csv(health_fitness_dataset)
    except FileNotFoundError:
        print(f"ОШИБКА: Файл не найден по пути {health_fitness_dataset}")
        return pd.DataFrame()

    # --- 1.1: Переименование (только нужных нам фич) ---
    df = df.rename(columns={
        'participant_id': 'user_id',
        'daily_steps': 'steps_total',
        'hours_sleep': 'sleep_hours_total',
        'calories_burned': 'calories_total',
        # Мы намеренно ИГНОРИРУЕМ 'avg_heart_rate'
    })

    # --- 1.2: Базовая обработка (типы, gender) ---
    df['date'] = pd.to_datetime(df['date'])
    df['gender_numeric'] = df['gender'].map({'M': 1, 'F': 0})

    # --- 1.3: Расчет производных фич (БЕЗ ПУЛЬСА) ---
    print("Расчет Z-scores и Deltas для сна и шагов...")
    df = df.sort_values(['user_id', 'date'])

    # Рассчитываем Z-оценки и Дельты за 7 дней
    df['z_sleep_7d'] = df.groupby('user_id', group_keys=False)['sleep_hours_total'].apply(_zscore)
    df['z_steps_7d'] = df.groupby('user_id', group_keys=False)['steps_total'].apply(_zscore)
    df['d_sleep_7d'] = df.groupby('user_id', group_keys=False)['sleep_hours_total'].apply(_delta_from_mean)
    df['d_steps_7d'] = df.groupby('user_id', group_keys=False)['steps_total'].apply(_delta_from_mean)
    
    # --- 1.4: КРИТИЧЕСКАЯ ОЧИСТКА (NaN) ---
    # Мы должны удалить NaN ДО создания цели, чтобы Z-оценки были чистыми
    features_to_check_na = ['z_sleep_7d', 'z_steps_7d', 'd_sleep_7d', 'd_steps_7d']
    start_len = len(df)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_cleaned = df.dropna(subset=features_to_check_na)
    print(f"Удалено {start_len - len(df_cleaned)} строк (период 'прогрева' Z-оценок).")

    # --- 1.5: Создание НОВОЙ цели 'y_target_fatigue' (Инжиниринг) ---
    # Мы ИГНОРИРУЕМ 'stress_level'. Мы создаем СВОЮ цель "Усталость".
    
    # НАШЕ ОПРЕДЕЛЕНИЕ "УСТАЛОСТИ" (1):
    # День, когда сон был ЗНАЧИТЕЛЬНО НИЖЕ нормы (z < -1.0)
    # ИЛИ активность была ЗНАЧИТЕЛЬНО ВЫШЕ нормы (z > 1.5)
    
    df_cleaned['y_target_fatigue'] = (
        (df_cleaned['z_sleep_7d'] < -1.0) |  # Сон на 1 std ниже нормы
        (df_cleaned['z_steps_7d'] >  1.5)    # Шаги на 1.5 std выше нормы
    ).astype(int)

    print("\n--- Баланс новой цели 'y_target_fatigue' ---")
    print(df_cleaned['y_target_fatigue'].value_counts(normalize=True))
    print("-------------------------------------------------")

    # --- 1.6: Выбор финальных столбцов ---
    FINAL_COLS_NEEDED = [
        'user_id',        # Для GroupShuffleSplit
        'y_target_fatigue', # Наша НОВАЯ цель
        
        # Фичи, которые будут у нас с телефона:
        'steps_total', 
        'calories_total', 
        'sleep_hours_total',
        'age', 
        'gender_numeric', 
        'height_cm', 
        'weight_kg',
        
        # Фичи, которые мы рассчитали:
        'z_sleep_7d',
        'z_steps_7d',
        'd_sleep_7d',
        'd_steps_7d'
    ]
    
    cols_exist = [col for col in FINAL_COLS_NEEDED if col in df_cleaned.columns]
    df_final = df_cleaned[cols_exist].copy()
    
    print("--- Очистка и создание фич завершены ---")
    return df_final


# ===================================================================
# --- ОСНОВНОЙ СКРИПТ (ВЫПОЛНЕНИЕ) ---
# ===================================================================

# === Шаг 0: Параметры ===

# --- ИСПРАВЬТЕ ЭТУ СТРОКУ (Ошибка 2) ---
# Укажите ПОЛНЫЙ путь к вашему файлу, если он не лежит в той же папке
CSV_PATH = "health_fitness_dataset.csv"  
# Например: "C:\\Users\\User\\Downloads\\health_fitness_dataset.csv"
# Или: "/Users/User/Downloads/health_fitness_dataset.csv"

EXPORT_DIR = Path("export")
EXPORT_DIR.mkdir(exist_ok=True)
ONNX_PATH = EXPORT_DIR / "fatigue_model_v1.onnx"
FEATURES_JSON = EXPORT_DIR / "fatigue_model_v1_features.json"
METRICS_JSON = EXPORT_DIR / "fatigue_model_v1_metrics.json"


# === Шаг 1: Запуск подготовки данных ===
print("=== Шаг 1: Запуск очистки и подготовки данных... ===")
df_ready = create_fatigue_model_dataset(CSV_PATH)


# === Шаг 2: Определение X, y, groups ===
FEATURES = [] 
if not df_ready.empty:
    print("\n=== Шаг 2: Определение X, y, и groups... ===")
    
    # Наш НОВЫЙ список фич (БЕЗ ПУЛЬСА)
    FEATURES = [
        # Сырые данные
        'steps_total', 
        'calories_total', 
        'sleep_hours_total',
        # Статика
        'age', 
        'gender_numeric', 
        'height_cm', 
        'weight_kg',
        # Производные (Дельты - это нормально, они не прямая утечка)
        'd_sleep_7d',
        'd_steps_7d'
    ]

    # .fillna(0.0) - безопасно для (age, gender, height...)
    X = df_ready[FEATURES].fillna(0.0).values
    y = df_ready['y_target_fatigue'].values # Наша новая цель
    groups = df_ready['user_id'].values

    print(f"--- Шаг 2: X, y, groups созданы. {len(FEATURES)} фич ---")
else:
    print("\nОШИБКА: 'df_ready' пуст. Выполнение остановлено.")


# === Шаг 3: Разделение на Train/Test (по пользователям) ===
if 'X' in locals() and len(X) > 0:
    print(f"\n=== Шаг 3: Разделение {len(np.unique(groups))} пользователей... ===")
    
    gss = GroupShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
    train_idx, test_idx = next(gss.split(X, y, groups=groups))

    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    print(f"Разделение завершено: {len(X_train)} train, {len(X_test)} test.")
else:
    print("\nОШИБКА: X не создан. Не могу разделить данные.")


# === Шаг 4: Обучение моделей ===
if 'X_train' in locals() and len(X_train) > 0:
    print("\n=== Шаг 4: Обучение моделей... ===")
    models = {}

    # --- 4.1 Logistic Regression (Baseline) ---
    print("Обучение 1/2: Logistic Regression...")
    logreg = Pipeline(steps=[
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(
            max_iter=2000,
            class_weight='balanced', # Используем 'balanced', т.к. цель может быть (70/30)
            random_state=42
        ))
    ])
    logreg.fit(X_train, y_train)
    proba_lr = logreg.predict_proba(X_test)[:, 1]
    pred_lr  = (proba_lr >= 0.5).astype(int)
    models['logreg'] = {
        "model": logreg,
        "metrics": {
            "acc": accuracy_score(y_test, pred_lr),
            "f1": f1_score(y_test, pred_lr),
            "auc": roc_auc_score(y_test, proba_lr),
            "precision": precision_score(y_test, pred_lr),
            "recall": recall_score(y_test, pred_lr),
        }
    }

    # --- 4.2 Random Forest (Main Model) ---
    print("Обучение 2/2: Random Forest...")
    rf = RandomForestClassifier(
        n_estimators=300, # 300 - хороший компромисс
        max_depth=None,
        min_samples_leaf=5, # 5 - для лучшего обобщения
        random_state=42,
        n_jobs=-1,
        class_weight='balanced_subsample' 
    )
    rf.fit(X_train, y_train)
    proba_rf = rf.predict_proba(X_test)[:, 1]
    pred_rf  = (proba_rf >= 0.5).astype(int)
    models['rf'] = {
        "model": rf,
        "metrics": {
            "acc": accuracy_score(y_test, pred_rf),
            "f1": f1_score(y_test, pred_rf),
            "auc": roc_auc_score(y_test, proba_rf),
            "precision": precision_score(y_test, pred_rf),
            "recall": recall_score(y_test, pred_rf),
        }
    }
    print("--- Обучение завершено. ---")
else:
    print("\nОШИБКА: X_train не найден. Обучение невозможно.")


# === Шаг 5: Выбор лучшей модели и экспорт ===
if 'models' in locals() and 'FEATURES' in locals():
    
    # --- 5.1 Выбор победителя ---
    best_name = max(models.keys(), key=lambda k: models[k]["metrics"]["auc"])
    best_model_data = models[best_name]

    print("\n=== Шаг 5: РЕЗУЛЬТАТЫ ===")
    print(f"🎉 Лучшая модель: {best_name.upper()}")
    print("Метрики на тестовых данных:")
    print(json.dumps(best_model_data["metrics"], indent=2))

    # --- 5.2 Экспорт в ONNX ---
    print(f"\nЭкспорт '{best_name}' в ONNX...")
    try:
        initial_type = [('input', FloatTensorType([None, len(FEATURES)]))]
        onx = convert_sklearn(
            best_model_data["model"], 
            initial_types=initial_type
        )
        with open(ONNX_PATH, "wb") as f:
            f.write(onx.SerializeToString())
        print(f"✅ Модель успешно сохранена: {ONNX_PATH.resolve()}")
    except Exception as e:
        print(f"❌ ОШИБКА экспорта в ONNX: {e}")

    # --- 5.3 Сохранение JSON-файлов для Backend ---
    try:
        with open(FEATURES_JSON, "w", encoding="utf-8") as f:
            json.dump({
                "name": "fatigue_risk_v1", # Новое имя!
                "version": "1.0.0",
                "features": FEATURES
            }, f, ensure_ascii=False, indent=2)
        print(f"✅ Порядок фич сохранен: {FEATURES_JSON.resolve()}")

        with open(METRICS_JSON, "w", encoding="utf-8") as f:
            json.dump({
                "best_model": best_name,
                **best_model_data["metrics"]
            }, f, ensure_ascii=False, indent=2)
        print(f"✅ Метрики сохранены: {METRICS_JSON.resolve()}")
    except Exception as e:
        print(f"❌ ОШИБКА сохранения JSON: {e}")
        
    print("\n--- Весь пайплайн успешно завершен! ---")

else:
    print("\nОШИБКА: 'models' или 'FEATURES' не найдены.")


=== Шаг 1: Запуск очистки и подготовки данных... ===
--- Загрузка сырых данных из health_fitness_dataset.csv ---
Расчет Z-scores и Deltas для сна и шагов...
Удалено 6005 строк (период 'прогрева' Z-оценок).

--- Баланс новой цели 'y_target_fatigue' ---
y_target_fatigue
0    0.796713
1    0.203287
Name: proportion, dtype: float64
-------------------------------------------------
--- Очистка и создание фич завершены ---

=== Шаг 2: Определение X, y, и groups... ===
--- Шаг 2: X, y, groups созданы. 9 фич ---

=== Шаг 3: Разделение 3000 пользователей... ===
Разделение завершено: 545520 train, 136176 test.

=== Шаг 4: Обучение моделей... ===
Обучение 1/2: Logistic Regression...


/var/folders/33/ngrwl821509g3d0y7t70z0300000gn/T/ipykernel_931/4178425072.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['y_target_fatigue'] = (


Обучение 2/2: Random Forest...
--- Обучение завершено. ---

=== Шаг 5: РЕЗУЛЬТАТЫ ===
🎉 Лучшая модель: RF
Метрики на тестовых данных:
{
  "acc": 0.9020899424274468,
  "f1": 0.7747994257241787,
  "auc": 0.9592008232426553,
  "precision": 0.7270191454291872,
  "recall": 0.8293018042448567
}

Экспорт 'rf' в ONNX...
✅ Модель успешно сохранена: /Users/aruuketurgunbaeva/health-monitoring-app/ml/export/fatigue_model_v1.onnx
✅ Порядок фич сохранен: /Users/aruuketurgunbaeva/health-monitoring-app/ml/export/fatigue_model_v1_features.json
✅ Метрики сохранены: /Users/aruuketurgunbaeva/health-monitoring-app/ml/export/fatigue_model_v1_metrics.json

--- Весь пайплайн успешно завершен! ---
